In [1]:
import pandas as pd

In [2]:
lecture_df = pd.read_csv("data/lectures.csv")
question_df = pd.read_csv("data/questions.csv")
response_df = pd.read_csv("data/responses.csv")
train_df = pd.read_csv("data/short_train.csv")

train_df.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


### Split Train Data

Split the train data on `content_type_id` (whether it was an exam or a lecture)

In [3]:
train_exam_filter = train_df['content_type_id'] == 0
train_lecture_filter = train_df['content_type_id'] == 1

train_exams_df = train_df[train_exam_filter]
train_lectures_df = train_df[train_lecture_filter]

Print a report

In [4]:
print('Exams', end='\n=====\n')
print(f'row count: {len(train_exams_df)}')
print(f'user count: {len(train_exams_df.groupby(["user_id"]))}')

print()

print('Lectures', end='\n========\n')
print(f'row count: {len(train_lectures_df)}')
print(f'user count: {len(train_lectures_df.groupby(["user_id"]))}')

Exams
=====
row count: 343298
user count: 1215

Lectures
row count: 6701
user count: 505


In [8]:
class User:
    def __init__(self, user_id, timestamp, answered_correctly):
        self.user_id = user_id
        self.timestamp = timestamp
        self.answered_correctly = answered_correctly
        

In [47]:
import json

file_name = './data/short_train.csv'
users = []
with open(file_name, 'r', encoding="utf-8") as in_file:
    for i, line in enumerate(in_file):

        # Skip headers
        if i == 0:
            continue

        line = line.split(',')

        # Skip lectures
        if line[4] == '1':
            continue

        timestamp = int(line[1])
        user_id = int(line[2])
        answered_correctly = int(line[7]) == 1

        users.append(User(user_id, timestamp, answered_correctly))




In [56]:
from sklearn.model_selection import train_test_split

training_data, test_data = train_test_split(users, test_size=0.33, random_state=42)

print(len(training_data))
print(len(test_data))

230009
113289


In [57]:
train_x = [user.user_id for user in training_data]
train_y = [user.timestamp for user in training_data]
train_z = [user.answered_correctly for user in training_data]

test_x = [user.user_id for user in test_data]
test_y = [user.timestamp for user in test_data]
test_z = [user.answered_correctly for user in test_data]

In [58]:
print(f'answered correctly: {train_z.count(True)}')
print(f'answered incorrectly: {train_z.count(False)}')

answered correctly: 153737
answered incorrectly: 76272


### Prediction
Determine the best machine learning model